In [ ]:
!pip install -r /kaggle/input/test-task-kvant-data/requirements-3.txt

In [ ]:
!pip install transformers==4.49.0
!pip uninstall -y numpy scipy
!pip install --upgrade --force-reinstall numpy==1.24.0 scipy

In [3]:
from datasets import load_dataset
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.docstore.document import Document
import os, json
import re
import difflib
import sqlite3

2025-07-06 11:20:47.560804: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751800847.756351      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751800847.811143      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")["passages"]
dataset_small = dataset.select(range(2000))

documents = []
for example in dataset_small:
    passage = example['passage']
    doc_id = str(uuid4())
    documents.append({'id': doc_id, 'text': passage})


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name='/kaggle/input/test-task-kvant-data/all-MiniLM-L6-v2/all-MiniLM-L6-v2')

vector_store = Chroma(
    collection_name="mini_wiki_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_db",
)

chunks = [Document(page_content=d['text'], metadata={"id": d['id']}) for d in documents]
ids = [d['id'] for d in documents]
vector_store.add_documents(documents=chunks, ids=ids)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/test-task-kvant-data/Llama-3.2-3B-Instruct/Llama-3.2-3B-Instruct')
model = AutoModelForCausalLM.from_pretrained('/kaggle/input/test-task-kvant-data/Llama-3.2-3B-Instruct/Llama-3.2-3B-Instruct')

generator_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
)

llm = HuggingFacePipeline(pipeline=generator_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipykernel_35/1309108278.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator_pipeline)


In [ ]:
!pip install evaluate rouge_score

In [8]:
DB_PATH = "chat_history.db"
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS chat_history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT NOT NULL,
    answer TEXT NOT NULL
)
''')
conn.commit()

def save_history(question, answer):
    conn = sqlite3.connect('chat_history.db')
    c = conn.cursor()
    c.execute("INSERT INTO chat_history (question, answer) VALUES (?, ?)", (question, answer))
    conn.commit()
    conn.close()

def extract_final_answer(full_output):
    import re
    match = re.search(r'Provide your final answer below:\s*(.*)', full_output, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    else:
        return full_output.strip()

def parse_answer_from_output(output):
    pattern = r"Provide your final answer below.*?\n([\s\S]+)"
    match = re.search(pattern, output, re.IGNORECASE)
    
    if match:
        answer_part = match.group(1).strip()
        for line in answer_part.splitlines():
            line = line.strip()
            if line:
                line = re.sub(r"📝 Answer:.*", "", line).strip()
                line = line.replace('"', '').replace("'", "").strip()
                if line.endswith('.'):
                    line = line[:-1].strip()
                line = line.lower()
                return line
        return answer_part
    else:
        output = output.strip()
        output = output.replace('"', '').replace("'", "").strip()
        if output.endswith('.'):
            output = output[:-1].strip()
        output = output.lower()  
        return output 
        
def rag_chat(question, k=3):
    retriever = vector_store.as_retriever(search_kwargs={'k': k})
    docs = retriever.invoke(question)
    knowledge = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""
    You are a highly knowledgeable and concise AI assistant.

    Your role is to help the user by answering their questions **clearly, accurately, and extremely concisely**, using ONLY the information provided to you in the context section below.
    
    Guidelines:
    - Provide a **direct and minimal answer** to the user's question, ideally **one word, a number, or a very short phrase**.
    - Do **NOT** provide long explanations or additional information.
    - Do **NOT** mention or refer to any context, documents, or sources you used for your answer.
    - Do **NOT** use any external or prior knowledge that is not included in the provided context.
    - Your tone should be **neutral, confident, and professional**.
    
    Here is the user’s question:
    
    {question}
    
    Here is the context you must use to answer:
    
    {knowledge}
    
    Remember: If the context does not contain sufficient information to answer the question, say **"I don't know."**
    
    Provide your final answer below (ONLY the answer, nothing else):

    """

    # print(f'KNOWLEGE >>>>>>>>>>> {knowledge}')
    
    raw_response = llm(prompt)
    

    if isinstance(raw_response, list) and len(raw_response) > 0 and 'generated_text' in raw_response[0]:
        raw_text = raw_response[0]['generated_text']
    elif isinstance(raw_response, str):
        raw_text = raw_response
    else:
        raw_text = str(raw_response)

    response = parse_answer_from_output(raw_text)

    print("MODEL ANSWER >>>>>>>>>>", response)

    save_history(question, response)
    return response

In [9]:
# query = "What is Uruguay?"
# answer = rag_chat(query)
# print("📝 Answer:", answer)

# c.execute("SELECT * FROM chat_history")
# rows = c.fetchall()
# for row in rows:
#     print(row)

# conn.close()

In [ ]:
import numpy as np
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.embeddings import HuggingFaceEmbeddings
import evaluate
dataset_qa = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
dataset_eval = dataset_qa["test"].select(range(100))  

rouge_metric = evaluate.load("rouge")

def exact_match(prediction, references):
    pred_norm = prediction.strip().lower()
    for ref in references:
        if pred_norm == ref.strip().lower():
            return 1
    return 0

def fuzzy_exact_match(prediction, references, threshold=0.8):
    pred_norm = prediction.strip().lower()
    for ref in references:
        ref_norm = ref.strip().lower()
        ratio = difflib.SequenceMatcher(None, pred_norm, ref_norm).ratio()
        if ratio >= threshold:
            return 1
    return 0

def normalize_case(text):
    if isinstance(text, list):
        return [t.lower().strip() for t in text]
    elif isinstance(text, str):
        return text.lower().strip()
    else:
        return text
        
def semantic_similarity(prediction, references):
    pred_emb = embedding_model.embed_query(prediction)
    ref_embs = [embedding_model.embed_query(ref) for ref in references]
    sims = cosine_similarity([pred_emb], ref_embs)
    return np.max(sims)

exact_matches = []
semantic_similarities = []
bleu_scores = []
rouge_scores = []

for example in dataset_eval:
    question = example["question"]
    references = normalize_case(example["answer"])
    print(f'EXAMPLE ANSWER >>>>>>>>>>>>> {references}')
    
    prediction = rag_chat(question)  
    
    em = fuzzy_exact_match(prediction, references, threshold=0.8)
    exact_matches.append(em)
    
    ss = semantic_similarity(prediction, references)
    semantic_similarities.append(ss)
    
    rouge = rouge_metric.compute(
        predictions=[prediction],
        references=[references]  
    )
    rouge_scores.append(rouge["rougeL"])  

print(f"Fuzzy Exact Match (EM) score: {np.mean(exact_matches):.3f}")
print(f"Semantic Similarity score: {np.mean(semantic_similarities):.3f}")
print(f"ROUGE-L score: {np.mean(rouge_scores):.3f}")